# QUBO Formulation for Quantum Credit Scoring

In the previous reports of the project, spcifically for the WP5, we defined the QUBO forumulation for the rating scale definition problem. The aim of the project is to divide $n$ counterparts in $m$ grades in accordance with several constraints. 

In this notebook we provide the toy model development of the cost function. Moreover, we solve the problem with different methods checking if the achieved solutions satisfy all the requested constraints.
Each part of the code is properly commented to provide a better understanding of its functionalities.

## Libraries and hyperparameters definition

In this section, we import all the Python libraries required for the execution of the code. Additionally we set the hyperparameters of the problem.

It should be emphasized that we implement the possibility to use random data or to import them from the database provided by Intesa Sanpaolo (not loaded into this repository).

There is also the possibility of selecting only certain constraints, both in the definition of the QUBO problem and in the verification of its solution.

In [12]:
import math
import time
import itertools
import dimod
import hybrid

In [13]:
config = {
    
    'random_data': 'yes',       # select 'yes' to generate random dataset
    'data_path': 'data/dataset-ispq.csv', # could be omitted to generate random data
    
    'n_counterpart': 6,         # could be a number or 'all'
    'grades': 2,
    'default_prob': 0.4,        # probability of default (range: 0 - 1)

    'attributes': {             # select optional attributes from the database
        'years': [2012, 2015],  # list of (not consecutive) years from 2012 to 2020, [] = ignore attribute
        'sector': 'no',
        'revenue': 'no',
        'geo_area': 'no',
    },

    # alpha parameters for hypothesis tests
    'alpha_concentration': 0.05,
    'alpha_heterogeneity': 0.01,
    'alpha_homogeneity': 0.05,

    # number of shots for dwave solver
    'shots': 100000,

    # enable / disable constraints
    'constraints': {
        'one_class': True,
        'logic': True,
        'conentration': True,
        'min_thr': True,
        'max_thr': True,
        'monotonicity': True,
    },
    
    # set the mu parameters for each contraint
    'mu': {
        'one_class': 100,
        'logic': 100,
        'concentration': 20,
        'min_thr': 10,
        'max_thr': 10,
        'monotonicity': 10,
    },

    # enable / disable tests
    'test': {
        'logic': True,
        'conentration': True,
        'min_thr': True,
        'max_thr': True,
        'monotonicity': True,
        'heterogeneity': False,
        'homogeneity': False,
    }
}

## Constraints definitions

Sections 1, 2 and 3 of the WP5 report contain the logical and the financial constraints description for the rating scale definition problem.

To recall the main points, we define a two-index binary variable:
$$
x_{ij} =
\begin{cases}
1 & \text{if the counterpart \(i\) is in the grade \(j\)} \\
0 & \text{otherwise}
\end{cases}
$$
that can be represented as elements of a matrix $x$ of size $n*m$, where $n$ is the number of counterparts and $m$ the number of grades. Then we define several constraints that must be fulfill to find the correct values of the matrix elements. The constraints are formulated as penalties of a QUBO problem. Solving the resulting QUBO formulation, it is possible to classify the 
$n$ counterparts into the $m$ grades.

Chapter 6 reports the mathematical definition of the constraints for the QUBO formulation. Here we report only a brief overview of them:
* **Logic constraint**: given the n counterparts ordered by score, the result of the QUBO problem must be a binary staircase matrix;
* **Monotonicity constraint**: the default rate of the grades increases as the grade index increases;
* **Heterogeneity constraint**: the default values of two consecutive grades must be distant from each other;
* **Concentration constraint**: the counterparts are divided into grades to avoid high concentrations of the default rate;
* **Grade cardinality threshold constraints**: the number of counterparts per grade is limited from above and below.

In the following sections we implement a function for each of them except for the heterogeneity one. Indeed, we considered not implementing the heterogeneity constraint in the cost function because the number of variables introduced to solve this constraint would make the entire problem intractable. The main reason behind this behavior is that the heterogeneity constraint we introduce in the section 6.3 of the WP5 report is of degree 24. Consequently, the corresponding quadratic term that enforces this requirement in the cost function introduces an exponential number of variables, which are necessary for its quadratization. As with the homogeneity test described in Chapter 4 of the WP5 report, we decided to implement only the test for the heterogeneity constraint.

We provide below a brief description of the code for each constraint, especially for those that required slight changes from what was described in the WP5 report.

### Logic constraint

The contribution of the logic constraint in the cost function (as reported in Appendix B) is:
$$
\begin{align} 
    P^{(0)}(x_{ij}) =& \mu_{01} \cdot (1-x_{11})\biggl(\sum_{j=2}^{m} x_{1j}\biggr)\\
    &+\mu_{02} \cdot (1-x_{nm})\biggl(\sum_{j=1}^{m-1} x_{nj}\biggr)\\
    &+\mu_{03} \sum_{i} \biggl(\sum_{j}x_{ij}-1\biggr)^{2}\\
    &+\mu_{04} \sum_{i,j=1}^{n-1,m-1} \Bigl[(1-x_{i+1,j}-x_{i+1,j+1})x_{ij}+x_{i+1,j}x_{i+1,j+1}\Bigr]\\
    &+\mu_{05} \sum_{i,j=1}^{n-1,m-1} \Bigl[(1-x_{ij}-x_{i,j+1})x_{i+1,j+1}+x_{ij}x_{i,j+1}\Bigr]\\
    &+\mu_{06} \sum_{i,j=1}^{n-1,m-1} x_{i,j+1}x_{i+1,j}\\
    &+\mu_{07} \sum_{i=1}^{n-1} \Bigl[(1-x_{ij}-x_{i,j+1})x_{i+1,j} + x_{ij}x_{i,j+1}\Bigr]
\end{align}
$$
where:
* $(1)$ expresses that the first counterpart must be in the first grade;
* $(2)$ expresses that the last counterpart must be in the last grade;
* $(3)$ expresses that one class must be in one and only one class;
* $(4)$, $(5)$ and $(6)$ avoid respectivly the submatrices:

$$
\begin{align*} 
\begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}, \quad
\begin{pmatrix}
0 & 0 \\
0 & 1
\end{pmatrix}, \quad
\begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix}
\end{align*};
$$

* $(7)$ avoids the following submatrix only for the first column (meaning that we penalize restarting from grade 0):
$$
\begin{align*} 
\begin{pmatrix}
0 & 0 \\
1 & 0
\end{pmatrix}
\end{align*};
$$

The following cell contains the functions that implement all these constraints.

We underline that penalties $1$ and $2$ are implemented in functions with descriptive names reflecting their purpose. These functions are called by `staircase_constr()`, which also implements penalties $4$, $5$, and $6$. For simplicity, $\mu_{01}=\mu_{02}=\mu_{04}=\mu_{05}=\mu_{06}=\mu_{07}$, whereas $\mu_{03}$ and its corresponding penalty are kept independent to allow greater flexibility in testing. 

In [14]:
# penalty: "first counterpart in first class"
def first_counterpart_const(m, n, mu=1):
    Q = np.zeros([n*m, n*m])
    
    for jj in range(1, m):
        Q[jj][jj] += 1
        Q[0][jj] -= 0.5
        Q[jj][0] -= 0.5
    return mu*Q

# penalty: "last counterpart in the last class"
def last_counterpart_const(m, n, mu=1):
    Q = np.zeros([n*m, n*m])

    for jj in range(m-1):
        tt = (n-1)*m+jj
        Q[tt][tt] += 1
        Q[(n*m)-1][tt] -= 0.5
        Q[tt][(n*m)-1] -= 0.5
    return mu*Q

# penalty: "one class per counterpart"
def one_class_const(m, n, mu=1):
    Q = np.zeros([n*m, n*m])
    c = 0

    for ii in range(n):
        for jj in range(m):
            tt = ii*m+jj
            Q[tt][tt] += -1
        for jj in range(m-1):
            for kk in range(jj+1,m):
                tt = ii*m+jj
                rr = ii*m+kk
                Q[tt][rr] += 1
                Q[rr][tt] += 1
        c += 1
    return (mu*Q, mu*c)

# penalty: "staircase matrix"
def staircase_constr(m, n, mu=1):
    Q = first_counterpart_const(m,n) + last_counterpart_const(m,n)

    # penalize not permitted submatrix, where a submatrix is
    # [[x1, x1], [x3, x4]]
    for ii in range(n-1):

        # penalize: [[1 0],[0 0]], [[0 0],[0 1]], [[0 1],[1 0]]
        for jj in range(m-1):
            x1 = ii*m+jj
            x2 = x1+1
            x3 = (ii+1)*m+jj
            x4 = x3+1

            # add linear terms
            Q[x1][x1] += 1
            Q[x4][x4] += 1

            # add quadratic terms
            Q[x1][x2] += 0.5
            Q[x2][x1] += 0.5

            Q[x1][x3] -= 0.5
            Q[x3][x1] -= 0.5

            Q[x1][x4] -= 1
            Q[x4][x1] -= 1

            Q[x2][x3] += 0.5
            Q[x3][x2] += 0.5

            Q[x2][x4] -= 0.5
            Q[x4][x2] -= 0.5

            Q[x3][x4] += 0.5
            Q[x4][x3] += 0.5

        # penalize restarting from class 0
        x1 = ii*m
        x2 = x1+1
        x3 = (ii+1)*m

        Q[x3][x3] += 1

        Q[x1][x3] -= 0.5
        Q[x3][x1] -= 0.5

        Q[x2][x3] -= 0.5
        Q[x3][x2] -= 0.5

        Q[x1][x2] += 0.5
        Q[x2][x1] += 0.5

    return mu*Q

### Monotonicity constraint

write me...

In [15]:
def monotonicity():
    
    return

### Concentration constraint

write me...

In [16]:
def concentration_constr(m, n, mu=1):
    Q = np.zeros([n*m, n*m])

    u = np.zeros([n * n * m, 2], dtype=int)
    index = 0
    for i1 in range(n):
        for i2 in range(n):
            for j in range(m):
                u[index] = [(i1)*m+j, (i2)*m+j]
                index += 1

    # penalty: "concentration"
    c = 1/(1-m)
    gamma = m/(m-1)
    for (u1, u2) in u:
        if u1==u2:
            Q[u1][u2] += gamma
        else:
            Q[u1][u2] += gamma/2

    return (mu*Q, mu*c)

### Grade cardinality threshold constraint

The contribution of the grade cardinality threshold constraints in the cost function (following appendix H) is:

$$
\begin{align}
    P^{(4)} _S (\vec{x},\vec{s}_{41},\vec{s}_{42})
    =&
    \mu_{41S} \cdot \biggl( \left\lceil\frac{n}{100}\right\rceil ^2 m +
    \sum_{u_1 u_2} x_{u_1} x_{u_2} + \sum_{j, l_1, l_2 =0}^{N_{S}^{(41)}-1} 2^{l_1} 2^{l_2} s_{l_1 j}^{(41)} s_{l_2 j}^{(41)}
    -2 \left\lceil\frac{n}{100}\right\rceil \sum_{u} x_{u}+ \nonumber\\
    &+ 2 \left\lceil\frac{n}{100}\right\rceil
    \sum_{j, l=0}^{N_{S}^{(41)}-1} 2^l s_{lj}^{(41)}-2\sum_{u} x_{u} \sum_{j, l=0}^{N_{S}^{(41)}-1} 2^l s_{lj}^{(41)} \biggr)  \tag{8} \\
    &+\mu_{42S}  \cdot \biggl( \left\lfloor\frac{15n}{100}\right\rfloor ^2 m + \sum_{u_1 u_2}x_{u_1}x_{u_2} + \sum_{j, l_1, l_2 =0}^{N_{S}^{(42)}-1} 2^{l_1} 2^{l_2} s_{l_1 j}^{(42)} s_{l_2 j}^{(42)}
    - 2 \left\lfloor\frac{15n}{100}\right\rfloor \sum_{u} x_{u} \nonumber\\
    &- 2 \left\lfloor\frac{15n}{100}\right\rfloor \sum_{j, l=0}^{N_{S}^{(42)}-1} 2^l s_{lj}^{(42)}
    +2 \sum_{u} x_{u} \sum_{j, l=0}^{N_{S}^{(42)}-1} 2^l s_{lj}^{(42)} \biggr). \tag{9} 
\end{align}
$$

The `threshold_constr()` function below implements both the lower and the upper threshold constraints, switching between one or the other depending on the `minmax` parameter. 

Here is important to notice that the first step of the function is computing the thresolds:
* the lower threshold is  1% of the counterparts or 1 if there are less than 100 counterparts
* the upper threshold is 15% of the counterparts but if there are less than 7 grades or if the 15% is less than 0, the upper threshold is set to $n-grades+1$. In these situations, there aren't any integer numbers such that the constraint is fulfilled.


In [17]:
def compute_lower_thrs(n):
    return math.floor(n*0.01) if math.floor(n*0.01) != 0 else 1

def compute_upper_thrs(n, grades):
    return math.floor(n*0.15) if grades > 7 and math.floor(n*0.15) != 0 else (n-grades+1)
    
def threshold_constr(m, n, offset, minmax, mu=1):

    if minmax == 'min':
        thr = compute_lower_thrs(n)
        slack_vars = math.floor(1+math.log2(n-thr)) # to check
    elif minmax == 'max':
        thr = compute_upper_thrs(n, m)
        slack_vars = math.floor(1+math.log2(thr)) # to check
    else:
        print("Error in threshold function call")
        sys.exit(1)

    # initialize Q and c
    dim = offset+slack_vars*m
    Q = np.zeros([dim, dim])
    c = m * thr * thr

    for i1 in range(n):
        for i2 in range(n):
            for j in range(m):
                u2 = [i1*m+j, i2*m+j]
                if u2[0]==u2[1]:
                    Q[u2[0]][u2[1]] += 1
                else:
                    Q[u2[0]][u2[1]] += 0.5
                    Q[u2[1]][u2[0]] += 0.5

    for l1 in range(slack_vars):
        for l2 in range(slack_vars):
            for j in range(m):
                v2 = [l1*m+j, l2*m+j]
                tmp = math.pow(2,math.floor((v2[0]+1)/m)+math.floor((v2[1]+1)/m))
                if v2[0]==v2[1]:
                    Q[offset+v2[0]][offset+v2[1]] += tmp
                else:
                    Q[offset+v2[0]][offset+v2[1]] += 0.5*tmp
                    Q[offset+v2[1]][offset+v2[0]] += 0.5*tmp


    for i in range(n):
        for j in range(m):
            u = i*m+j
            Q[u,u] -= 2*thr

    index = 0
    for l in range(slack_vars):
        for j in range(m):
            Q[offset+index][offset+index] += thr*math.pow(2,1+math.floor((l*m+j+1)/m))
            index += 1

    for i in range(n):
        for l in range(slack_vars):
            for j in range(m):
                w2 = [i*m+j, l*m+j]
                tmp = math.pow(2,1+math.floor((w2[1]+1)/m))
                Q[w2[0]][offset+w2[1]] -= -0.5*tmp
                Q[offset+w2[1]][w2[0]] -= -0.5*tmp

    return (mu*Q, mu*c)

## Test the constraints

In the file `check_constraint.py` we implemented the tests for all the constraints described in the report: given a matrix and the appropriate iperparameters, one function per constraint tests if that matrix fulfill that specific requirement.

All the functions are properly commented, so we refer the reader directly to the code for further details.

In [18]:
from src.check_constraints import *

## The solvers of the QUBO formulation

In order to properly classify the counterparts in the grades, we must solve the QUBO formulation. This means we must find the $x$ vector that minimizes the *cost function*:
$$
C(x) = x^t Q x + c .
$$
To accomplish this, we implemented several functions that explore different approaches.

First of all, the `brute_force_solver` function tests all the possible solutions of the system by comparing the value of the cost function for each of them. Since the $x$ vector has dimensions of $m*n$, we must test $2^{m*n}$ possible solutions. This means that this function is useful only for testing very small instances of our problem.

In [19]:
def brute_force_solver(Q, c, dim):

    # compute C(Y) = (Y^T)QY + (G^T)Y + c for every Y
    Ylist = list(itertools.product([0, 1], repeat=dim))
    Cmin = float('inf')

    for ii in range(len(Ylist)):
        Y = np.array(Ylist[ii])
        Cy=(Y.dot(Q).dot(Y.transpose()))+c
        if ( Cy < Cmin ):
            Cmin = Cy
            Ymin = Y.copy()
    
    return (np.array(Ymin), Cmin)

The second approach uses the functions provided by the D-Wave Ocean libraries. Specifically, the `exact_solver` function relies on the [dimod library](https://docs.dwavequantum.com/en/latest/ocean/api_ref_dimod/), where the `ExactSolver` method is implemented. Once a QUBO problem has been properly formatted, this method calculates the energy of all possible samples. However, it is very slow and it is therefore only suggested for testing small problem instances. Attempting to use it with larger instances results in an error related to insufficient memory to complete execution.

The input format required by the library is a Python dictionary, in which the keys represent the indices of the elements of the Q matrix, while the values correspond to the matrix entries themselves. This representation is known as a *binary quadratic model* (BQM).

Since the next solver requires an object of type `BinaryQuadraticModel`, we developed a function that generates the corresponding *dimod* object given the $Q$ matrix and the constant $c$ of the QUBO problem.

In [20]:
def from_matrix_to_bqm(matrix, c):
    
    Q_dict = {(i, j): matrix[i, j] for i in range(matrix.shape[0]) for j in range(matrix.shape[1])}# if matrix[i, j] != 0}
    bqm = dimod.BinaryQuadraticModel.from_qubo(Q_dict, c)

    return bqm

def exact_solver(bqm):
    
    sampler = dimod.ExactSolver()
    sampleset = sampler.sample(bqm)

    return sampleset

Until now, we have presented exact approaches with an exponential complexity. In the next cell, we will use another method from the Ocean D-Wave libraries to exploit the features of quantum computing: the `SimulatedAnnealingProblemSampler`.

As its name suggests, this method simulates the behavior of a quantum annealer to find the solution to a BQM. To obtain a solution, the following parameters are required: an initial state and the number of *shots*, i.e., the number of iterations to attempt before returning the best solution found up to that point.
Therefore, the solution returned by this function may not fully satisfy all the constraints of the problem. However, the advantage of using this method is that its computational complexity is linear with respect to the problem size.

It is important to note that D-Wave provides an analogous function that can be run directly on its quantum annealers. Due to limited access to such hardware, it is preferable to first validate the code using classical quantum simulators. For the purposes of this notebook, we therefore used the `SimulatedAnnealingProblemSampler` function to verify that the implemented algorithm actually produces acceptable results.

In [21]:
def annealer_solver(dim, bqm, shots):

    # define the initial state (all elements = 0 or random elements)
    state = hybrid.core.State.from_sample({i: 0 for i in range(dim)}, bqm)
    # state = hybrid.core.State.from_sample({i: np.random.randint(0, 2) for i in range(dim)}, bqm)

    sampler = hybrid.samplers.SimulatedAnnealingProblemSampler(num_sweeps=shots)
    result_state = sampler.run(state).result()
 
    return result_state

Write me...

In [22]:
import gurobipy as gpy
from gurobipy import GRB

def gurobi_solver(m, n, matrix, c, gurobi_n_sol, gurobi_fidelity):
    size = matrix.shape[0]
    # model definition
    qubo_model = gpy.Model("QCS")
    qubo_vars = qubo_model.addVars(size, vtype=GRB.BINARY, name="x")

    # cost function definition
    qubo_expr = gpy.QuadExpr()
    row_idxs, col_idxs = np.nonzero(matrix)
    for ii, jj in zip(row_idxs, col_idxs):
        qubo_expr.add(matrix[ii, jj] * qubo_vars[ii] * qubo_vars[jj])
    qubo_expr.addConstant(c)

    # add const function to the model
    qubo_model.setObjective(qubo_expr, GRB.MINIMIZE)

    # Setting solver parameters
    qubo_model.setParam("OutputFlag", 1) # verbosity
    qubo_model.setParam("Seed", 0)  # fix seed
    # qubo_model.setParam("TimeLimit", timelimit)
    
    # Search more than 1 solution
    num_max_solutions = gurobi_n_sol
    if num_max_solutions > 1:
        qubo_model.setParam("PoolSolutions", num_max_solutions)
        qubo_model.setParam("PoolSearchMode", 2)
        qubo_model.setParam("PoolGap", gurobi_fidelity)

    # Run the Gurobi QUBO optimization
    qubo_model.optimize()

    # Print result
    if qubo_model.Status in {GRB.OPTIMAL, GRB.SUBOPTIMAL}:
        if num_max_solutions == 1:
            solution = [int(qubo_vars[i].X) for i in range(size)]
            if len(solution) > m*n:
                solution = solution[:m*n]
            print("\nBest solution:\n", np.array(solution).reshape(n, m))
            print("Cost of the function:", qubo_model.ObjVal)
        else:
            # select all the solutions or num_max_solutions solutions
            nfound = min(qubo_model.SolCount, num_max_solutions)

            for sol_idx in range(nfound):
                qubo_model.setParam(GRB.Param.SolutionNumber, sol_idx)
                qubo_bitstring = np.array(
                    [int(qubo_vars[jj].Xn) for jj in range(size)]
                )
                if qubo_bitstring.shape[0] > m*n:
                    qubo_bitstring = qubo_bitstring[:m*n]
                print(f"solution {sol_idx+1}:\n{qubo_bitstring.reshape(n,m)}")
                print("Cost of the function:", qubo_model.PoolObjVal)
    else:
        print("No solutions found")

## Definition and resolution of a problem instance

Now that we have defined all the building blocks of our problem, we can create an instance of the problem and solve it using one or more of the methods introduced earlier. Then, we can verify that the solution meets all the problem's constraints.

The following code instantiates a dataset using the hyperparameters specified at the beginning of the notebook. Specifically, we can choose to generate a random dataset or read the counterparts from the one provided by Intesa Sanpaolo.

It is important to note that in both cases, the dataset is ordered by score: the counterpart with the lowest score is first and the counterpart with the highest score is last. This is a mandatory requirement for the algorithm.

After selecting the dataset, we build the QUBO formulation of that instance. We computed the $Q$ matrix and the constant $c$ by calling all the constraint functions that were defined before. For testing purposes, we implemented the choice of which constraints to implement using the appropriate hyperparameters in the `constraint` dictionary.

Finally, we convert the NumPy matrix $Q$ into the corresponding `BinaryQuadraticModel` object.




In [23]:
from src.select_data import *

# generate a random dataset 
dataset = generate_data(config) if config['random_data'] == 'yes' else load_data(config)
n = len(dataset)
m = config['grades']
default = dataset['default'].to_numpy().reshape(n,1)

# generate the appropriate Q matrix
start_time = time.perf_counter_ns()
Q = np.zeros([m*n, m*n])
c = 0
if config['constraints']['one_class'] == True:
    (Q_one_class,c_one_class) = one_class_const(m,n,config['mu']['one_class'])
    Q = Q + Q_one_class
    c = c + c_one_class
if config['constraints']['logic'] == True:
    Q = Q + staircase_constr(m,n,config['mu']['logic'])
if config['constraints']['conentration'] == True:
    (Q_conc,c_conc) = concentration_constr(m, n, config['mu']['concentration'])
    Q = Q + Q_conc
    c = c + c_conc
if config['constraints']['min_thr'] == True:
    (Q_min_thr, c_min_thr) = threshold_constr(m, n, Q.shape[0], 'min', config['mu']['min_thr'])
    pad = Q_min_thr.shape[0] - Q.shape[0]
    Q = np.pad(Q, pad_width=((0,pad), (0, pad)), mode='constant', constant_values=0) + Q_min_thr
    c = c + c_min_thr
if config['constraints']['max_thr'] == True:
    (Q_max_thr, c_max_thr) = threshold_constr(m, n, Q.shape[0], 'max', config['mu']['max_thr'])
    pad = Q_max_thr.shape[0] - Q.shape[0]
    Q = np.pad(Q, pad_width=((0,pad), (0, pad)), mode='constant', constant_values=0) + Q_max_thr
    c = c + c_max_thr
end_time = time.perf_counter_ns()

# generate the BMQ
bqm = from_matrix_to_bqm(Q, c)

print(f"Matrix size:{Q.shape}")
print(f"Time of generation: {(end_time - start_time)/10e9} s")

Matrix size:(24, 24)
Time of generation: 6.66848e-05 s


The next cells try to solve the QUBO problem using the solvers defined above:
* with the brute force solver
* with the exact solver by D-Wave
* with the annealing solver by D-Wave
* with gurobi

The tests were made for a very small instance with the only demostrative purpose. 

In [24]:
# Solving with brute force
start_time = time.perf_counter_ns()
(result_bf, cost) = brute_force_solver(Q,c,Q.shape[0])
end_time = time.perf_counter_ns()
if config['constraints']['min_thr'] == True:
    result_bf = result_bf[:m*n]
print(f"\nBrute Force result:\n{result_bf.reshape(n,m)}")
print(f"Time of brute force solution: {(end_time - start_time)/10e9} s\n")


Brute Force result:
[[1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 0]
 [0 0]]
Time of brute force solution: 4.3002426417 s



In [25]:
# Solving exactly with dwave
start_time = time.perf_counter_ns()
e_result = exact_solver(bqm)
df_result = e_result.lowest().to_pandas_dataframe()
end_time = time.perf_counter_ns()
elapsed_time_ns = end_time - start_time
# Print all the solutions
result_exact_solver = df_result.iloc[:, :m*n].to_numpy()
# print(f"All exact solutions:\n{df_result}")
print(f"Exact solutions with dwave: {int(result_exact_solver.size/(m*n))}")
for sol in result_exact_solver[:]:
    print(f"solution:\n{sol.reshape(n, m)}")
print(f"Time of all exact solutions: {elapsed_time_ns/10e9} s")
# print(f"First solution:\n{result_exact_solver[0].reshape(n, m)}")

Exact solutions with dwave: 4
solution:
[[1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 0]
 [0 0]]
solution:
[[1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 0]]
solution:
[[1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 0]]
solution:
[[1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]]
Time of all exact solutions: 1.5538667409 s


In [24]:
# Solving with annealing 
start_time = time.perf_counter_ns()
result = annealer_solver(Q.shape[0], bqm, config['shots'])
end_time = time.perf_counter_ns()
result_ann = np.array([int(x) for x in result.samples.first.sample.values()])[:m*n]
annealing_matrix = result_ann.reshape(n, m)
print(f"\nAnnealing result:\n{annealing_matrix}")    
print(f"\nTime of annealing solution: {(end_time - start_time)/10e9} s\n")

print("Result validation:")
verbose = True
dataset = generate_data(config) if config['random_data'] == 'yes' else load_data(config)
default = dataset['default'].to_numpy().reshape(n,1)

check_staircase(annealing_matrix, verbose)
check_concentration(annealing_matrix, config['alpha_concentration'], verbose)
check_lower_thrs(annealing_matrix, compute_lower_thrs(n), verbose)
check_upper_thrs(annealing_matrix, compute_upper_thrs(n,m), verbose)
check_heterogeneity(annealing_matrix, default, config['alpha_heterogeneity'], verbose)
check_homogeneity(annealing_matrix, default, config['alpha_homogeneity'], verbose)
print()


Annealing result:
[[1 0]
 [1 0]
 [0 1]
 [0 1]]

Time of annealing solution: 0.0024843874 s

Result validation:
	✓ Logic constraint checked
	✓ Concentration constraint checked
	✓ Lower threshold limit constraint checked
	✓ Upper threshold limit constraint checked
	x Error: heterogeneous constraint not respected
		 Grades 0 and 1 are not heterogeneous
	x Error in homogeneity constraint: at least one grade has less than 2 elements



In [25]:
gurobi_n_sol = 1
gurobi_fidelity = 1
gurobi_solver(m, n, Q, c, gurobi_n_sol, gurobi_fidelity)

Set parameter Username
Set parameter LicenseID to value 2679405
Academic license - for non-commercial use only - expires 2026-06-18
Set parameter OutputFlag to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 0 rows, 16 columns and 0 nonzeros
Model fingerprint: 0xd0010232
Model has 71 quadratic objective terms
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 580.0000000
Found heuristic solution: objective 260.0000000
Presolve removed 0 rows and 8 columns
Presolve time: 0.00s
Presolved: 19 rows, 27 columns, 